In [22]:
import pandas as pd
import os
  
import seaborn as sns
import plotly.express as px
import plotly.graph_objs as go
from plotly.tools import FigureFactory as FF
import re
import os
from datetime import datetime
os.getcwd()
df = pd.DataFrame()

In [7]:
os.listdir('./outb_data')

['18.xlsx', '2019.xlsx', '2020.xlsx', '21.xlsx', 'out17_fas.xlsx']

In [8]:
# %%time 
# pd.read_excel('./outb_data/2019.xlsx')

In [9]:
# %%time 
# os.listdir('./outb_data')

# for i in os.listdir('./outb_data'):
#     df = pd.concat([df, pd.read_excel('./outb_data/' + i)],\
#         axis = 0)

Wall time: 3min 40s


In [28]:
# df.to_csv('./out_full.csv', index = False)
df = pd.read_csv('./out_full.csv')

C:\Users\dscshap3808\Miniconda3\envs\siming\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning:

Columns (8,26,36,40,41,43,44,56,64,67,75,86,120,121,122,144) have mixed types.Specify dtype option on import or set low_memory=False.



In [29]:
df = df.dropna(how = 'all', axis = 1)
df = df.fillna(method = 'ffill')
df.columns = [i.upper() for i in df.columns.to_numpy()]


In [32]:
df.columns[df.columns.str.contains(r'DATE')]

Index(['TRAILING_STS_DATE', 'CONSOLIDATED', 'LEADING_STS_DATE',
       'DATE_TIME_STAMP', 'SCHEDULED_SHIP_DATE', 'PLANNED_SHIP_DATE',
       'ACTUAL_SHIP_DATE_TIME', 'CREATION_DATE_TIME_STAMP'],
      dtype='object')

In [49]:
df['ACTUAL_SHIP_DATE_TIME'].str.slice(11,).astype(str)

0        01:34:04.010
1        01:11:44.153
2        01:33:07.417
3        02:06:07.540
4        01:33:07.843
             ...     
69783    01:53:45.117
69784    01:53:45.430
69785    08:47:32.797
69786    08:47:33.237
69787    05:20:25.877
Name: ACTUAL_SHIP_DATE_TIME, Length: 69788, dtype: object

In [43]:
def date_mutate(col, df):
    df[col + '_TIME'] = df[col].str.slice(11,).astype(str)
    df[col] = df[col].str.slice(0,10)
    df[col + '_2'] = [datetime.strptime(i,'%H:%M:%S').strftime('%H%M%S') \
        for i in df[col + '_TIME']]
    return df.head(1)

for i in df.columns[df.columns.str.contains(r'date|DATE')].to_numpy():
    date_mutate(i, df)


first_df = df.groupby(
    'CREATION_DATE_TIME_STAMP'
    )['CREATION_DATE_TIME_STAMP_2'].min().reset_index();first_df


first_df = first_df.merge(
    df, 
    on = ['CREATION_DATE_TIME_STAMP', 'CREATION_DATE_TIME_STAMP_2'], 
    how = 'inner')

ValueError: time data '' does not match format '%H:%M:%S'

In [ ]:
def data_mani(first_df, df):
    train_first = first_df.groupby(
        'CREATION_DATE_TIME_STAMP'
        ).agg({'RECEIPT_ID':['nunique'], 
        'TOTAL_LINES': ['sum'], 
        'TOTAL_CONTAINERS': 'sum', 
        'TOTAL_WEIGHT':'sum',
        'TOTAL_QTY': ['sum'],
        'CREATION_DATE_TIME_STAMP_2': 'min' }).reset_index()
        
    train_full = df.groupby(
        'CREATION_DATE_TIME_STAMP'
        ).agg({'RECEIPT_ID':['nunique'], 
        'TOTAL_LINES': ['sum'], 
        'TOTAL_CONTAINERS': 'sum', 
        'TOTAL_WEIGHT':'sum',
        'TOTAL_QTY': ['sum'],
        'CREATION_DATE_TIME_STAMP_2': 'max' }).reset_index()

    train_full.columns = ['CREATION_DATE_TIME_STAMP','id', 'line_sum', 'container_sum', 'weight', 'qty', 'creat_time']
    train_first.columns = ['CREATION_DATE_TIME_STAMP','id', 'line_sum', 'container_sum', 'weight', 'qty', 'creat_time']

    train_first.index= pd.to_datetime(train_first['CREATION_DATE_TIME_STAMP'], \
        yearfirst = True,  format='%Y-%m-%d')
    train_full.index = train_full['CREATION_DATE_TIME_STAMP']
    train_full.index = pd.DatetimeIndex(train_full.index)

    train_full['CREATION_DATE_TIME_STAMP'] = pd.to_datetime(train_full['CREATION_DATE_TIME_STAMP'])
    train_first['CREATION_DATE_TIME_STAMP'] = pd.to_datetime(train_first['CREATION_DATE_TIME_STAMP'])
    return train_first  , train_full 



In [ ]:
train_first, train_full = data_mani(first_df, df)